In [9]:
import pandas as pd
import sqlite3
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import csv

import seaborn as sns
import matplotlib.pyplot as plt

# Import XGBoost classifier
from xgboost import XGBClassifier

# Import scikit-learn functions
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Import scikit-plot functions
from scikitplot.metrics import plot_roc_curve
from scikitplot.metrics import plot_precision_recall_curve
from scikitplot.metrics import plot_calibration_curve

# Import SciPy function
from scipy.spatial import distance

pd.set_option("display.max_columns", None)

In [10]:
# Data Directory
data_dir = "../../Data/Big-Data-Cup-2021"
bucketless_data_dir = '../../Data/bdc/data'
tracking_data_dir = "{}/TrackingData".format(data_dir)
db_path = '/Users/keltim01/Documents/databases/'

#connect to database 
conn = sqlite3.connect(db_path + 'bdc_2022.db')
cursor = conn.cursor()

## Using tracking data to find a way to classify teams in creating "danger" on the powerplay by creating space. 

- with tracking data get totally away from shots as basis of structure but take locations of players as structure → also maybe get away from shots as basis at all
- xG everywhere not shots to be able to classify? 
- Combine tracking and event data to look at nearest defender for attackers on the powerplay
- factor in shot selection: xG shoot now, xG Pass etc.
- BART model?

Important questions: 
- Who is good in creating space? 
- Who creates more dangerous situations (more xG)? 
- Who creates danger and space in formation? 
- Who uses good chances (finishing) or selects good shots?  

In [11]:
# Merging Tracking and Play-by-Play data 
pd.read_sql('SELECT * FROM tracking', conn).head()

,index,frame_id,period,track_id,team_id,team_name,jersey_number,x_coord,y_coord,video_shot,game_seconds,venue,player,position
0,0,96,1,2,Light,ROC,100,176.651307,52.308133,1,2590,away,None,None
1,1,96,1,3,Light,ROC,27,158.782222,41.325801,1,2590,away,Veronika Korzhakova,Center
2,2,96,1,4,Dark,Finland,10,151.138502,51.883526,1,2590,home,Elisa Holopainen,Left Wing
3,3,96,1,5,Light,ROC,100,163.814533,59.253376,1,2590,away,None,None
4,4,96,1,8,Dark,Finland,88,165.000000,50.000000,1,2590,home,Ronja Savolainen,Defense
